In [1]:
import torch
import torch.nn.functional as F

def compute_yolo_loss(preds, targets, S=16, B=2, num_classes=3, lambda_coord=5.0, lambda_noobj=0.5, lambda_area=1.0):
    """
    YOLO形式のマルチタスク損失関数（本番用）
    - バウンディングボックス回帰（cx, cy, w, h）
    - クラス分類（クロスエントロピー）
    - 面積回帰（回帰損失）
    - objectness（二値分類）

    Args:
        preds: Tensor [B, S, S, B, 1 + 4 + num_classes + 1]
               モデルの出力
        targets: list[list]（各バッチごとのラベルリスト）
               各要素は [class_id, cx, cy, w, h, area]
        S: 出力グリッド数（S × S）
        B: 各グリッドセルあたりの予測ボックス数
        num_classes: 図形クラス数（円・三角形・四角形 → 3）
        lambda_coord: bbox回帰の重み（デフォルト: 5.0）
        lambda_noobj: objectnessが0のセルの損失重み（デフォルト: 0.5）
        lambda_area: 面積回帰損失の重み（デフォルト: 1.0）

    Returns:
        総合損失（スカラー）
    """
    
    device = preds.device
    batch_size = preds.shape[0]
    loss = 0.0  # 総損失初期化

    for b in range(batch_size):
        # 1バッチ分の空ラベルテンソルを準備（予測と同形式）
        target_tensor = torch.zeros((S, S, B, 1 + 4 + num_classes + 1), device=device)

        for t in targets[b]:
            # クラス・位置・サイズ・面積を取得
            class_id, cx, cy, w, h, area = t
            class_id = int(class_id)

            # 対応するグリッドセル座標を取得（整数）
            grid_x = int(cx * S)
            grid_y = int(cy * S)

            # セル内での相対座標（0〜1）
            gx = cx * S - grid_x
            gy = cy * S - grid_y

            if grid_x >= S or grid_y >= S:
                continue  # 画像外には登録しない（保険）

            # B個の枠のうち、最初の空きスロットに登録
            for box in range(B):
                if target_tensor[grid_y, grid_x, box, 0] == 0:
                    # objectness = 1
                    target_tensor[grid_y, grid_x, box, 0] = 1.0

                    # bbox座標（セル内）
                    target_tensor[grid_y, grid_x, box, 1:5] = torch.tensor([gx, gy, w, h], device=device)

                    # クラス one-hot（例: [0,1,0]）
                    target_tensor[grid_y, grid_x, box, 5 + class_id] = 1.0

                    # 面積
                    target_tensor[grid_y, grid_x, box, -1] = area
                    break

        # 対象バッチの予測とラベルを取得
        pred = preds[b]     # [S, S, B, D]
        target = target_tensor

        # objectness マスク（物体あり / なし）
        obj_mask = target[..., 0] == 1
        noobj_mask = target[..., 0] == 0

        # ---------- 損失計算 -----------

        # 1. objectness（二値分類）: BCE Loss
        bce_obj = F.binary_cross_entropy_with_logits(
            pred[..., 0], target[..., 0], reduction='none'
        )
        loss += (bce_obj * obj_mask).sum() + lambda_noobj * (bce_obj * noobj_mask).sum()

        # 2. bbox（回帰）: MSE Loss（物体がある場所だけ）
        if obj_mask.any():
            loss += lambda_coord * F.mse_loss(
                pred[..., 1:5][obj_mask],
                target[..., 1:5][obj_mask],
                reduction='sum'
            )

            # 3. クラス分類: Cross Entropy（物体がある場所だけ）
            pred_cls = pred[..., 5:5+num_classes][obj_mask]               # [N, C]
            target_cls = target[..., 5:5+num_classes][obj_mask]           # [N, C]
            target_cls_ids = target_cls.argmax(dim=-1)                    # [N]
            loss += F.cross_entropy(pred_cls, target_cls_ids, reduction='sum')

            # 4. 面積回帰: MSE Loss（物体がある場所だけ）
            pred_area = pred[..., -1][obj_mask]
            target_area = target[..., -1][obj_mask]
            loss += lambda_area * F.mse_loss(pred_area, target_area, reduction='sum')

    return loss / batch_size  # バッチ平均で正規化